In [1]:
%load_ext autoreload

In [18]:
%autoreload 
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML
from utilities import save_model as save

ImportError: cannot import name 'save_model' from 'h2o.automl' (/home/henrikraa/anaconda3/envs/h2o/lib/python3.10/site-packages/h2o/automl/__init__.py)

In [3]:
h2o.init(max_mem_size='10G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "17.0.6" 2023-01-17 LTS; Java(TM) SE Runtime Environment (build 17.0.6+9-LTS-190); Java HotSpot(TM) 64-Bit Server VM (build 17.0.6+9-LTS-190, mixed mode, sharing)
  Starting server from /home/henrikraa/anaconda3/envs/h2o/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmptdzu7pw3
  JVM stdout: /tmp/tmptdzu7pw3/h2o_henrikraa_started_from_python.out
  JVM stderr: /tmp/tmptdzu7pw3/h2o_henrikraa_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_henrikraa_oxxj6y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [4]:
data_path = 'data_processed/'


In [5]:
train = pd.read_csv(f'{data_path}/train_mean.csv')

train = train.set_index('date_forecast')
train.index = pd.to_datetime(train.index)

In [6]:
train.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,...,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,pv_measurement
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,7.700,1.22825,0.00,0.00,0.0,280.300,0.000,0.000,0.00,0.000,...,-3.77425,0.0,286.225,100.000,40386.477,3.600,-3.575,-0.500,0.0,0.00
2019-06-02 23:00:00,7.700,1.22350,0.00,0.00,0.0,280.300,0.000,0.000,0.00,0.000,...,-4.35725,0.0,286.900,100.000,33770.650,3.350,-3.350,0.275,0.0,0.00
2019-06-03 00:00:00,7.875,1.21975,0.00,0.00,0.0,280.650,0.000,0.000,0.00,0.000,...,-3.30950,0.0,286.950,100.000,13595.500,3.050,-2.950,0.750,0.0,0.00
2019-06-03 01:00:00,8.425,1.21800,208.65,0.75,0.0,281.675,0.300,526.775,0.00,0.000,...,-0.82250,0.0,286.750,100.000,2321.850,2.725,-2.600,0.875,0.0,0.00
2019-06-03 02:00:00,8.950,1.21800,32468.15,23.10,0.0,282.500,11.975,22068.950,0.15,282.975,...,3.05125,0.0,286.450,99.225,11634.800,2.550,-2.350,0.925,0.0,19.36


In [7]:
X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
autoML = H2OAutoML(max_models=30, seed=5)
autoML.train(x=X, y=y, training_frame=X_train)

AutoML progress: |


17:39:02.216: XGBoost_1_AutoML_2_20231027_173902 [XGBoost def_2] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_1_AutoML_2_20231027_173902_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


██████████
17:40:12.897: XGBoost_2_AutoML_2_20231027_173902 [XGBoost def_1] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_2_AutoML_2_20231027_173902_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


███████████
17:45:30.650: XGBoost_3_AutoML_2_20231027_173902 [XGBoost def_3] failed: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for XGBoost model: XGBoost_3_AutoML_2_20231027_173902_cv_1.  Details: ERRR on field: _response_column: Response contains missing values (NAs) - not supported by XGBoost.


███████

key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/30
# GBM base models (used / total),5/14
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/13
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [15]:
best_model = autoML.leader
best_model
# save.save_model_to_local_disk(best_model, '27E_h2o_stacked_esemble_mean_preprocessing')

key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/30
# GBM base models (used / total),5/14
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/13
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [19]:
# save the model
h2o.save_model(best_model, path = "/home/henrikraa/models/", force=True)

'/home/henrikraa/models/StackedEnsemble_AllModels_1_AutoML_2_20231027_173902'